Note the cells that check whether lambda is correct are out of date because Dominic changed the lambdas after I wrote thie notebook

In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=6)
           2	LOAD_GLOBAL(arg=0, lineno=6)
           4	LOAD_METHOD(arg=1, lineno=6)
           6	LOAD_CONST(arg=1, lineno=6)
           8	CALL_METHOD(arg=1, lineno=6)
          10	STORE_FAST(arg=2, lineno=6)
          12	LOAD_FAST(arg=0, lineno=7)
          14	LOAD_CONST(arg=2, lineno=7)
          16	BINARY_SUBSCR(arg=None, lineno=7)
          18	LOAD_FAST(arg=1, lineno=7)
          20	LOAD_CONST(arg=3, lineno=7)
          22	BINARY_SUBSCR(arg=None, lineno=7)
          24	BINARY_MULTIPLY(arg=None, lineno=7)
          26	LOAD_FAST(arg=0, lineno=7)
          28	LOAD_CONST(arg=3, lineno=7)
          30	BINARY_SUBSCR(arg=None, lineno=7)
          32	LOAD_FAST(arg=1, lineno=7)
          34	LOAD_CONST(arg=2, lineno=7)
          36	BINARY_SUBSCR(arg=None, lineno=7)
          38	BINARY_MULTIPLY(arg=None, lineno=7)
          40	BINARY_SUBTRACT(arg=None, lineno=7)
          42	LOAD_FAST(arg=2, lineno=7)
          44	LO

DEBUG:numba.core.byteflow:dispatch pc=74, inst=BINARY_MULTIPLY(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$72binary_subscr.33']
DEBUG:numba.core.byteflow:dispatch pc=76, inst=BINARY_SUBTRACT(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$74binary_multiply.34']
DEBUG:numba.core.byteflow:dispatch pc=78, inst=LOAD_FAST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35']
DEBUG:numba.core.byteflow:dispatch pc=80, inst=LOAD_CONST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36']
DEBUG:numba.core.byteflow:dispatch pc=82, inst=STORE_SUBSCR(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36', '$const80.37']
DEBUG:numba.core.byteflow:dispatch pc=84, inst=LOAD_FAST(arg=0, lineno=9)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=86, inst=LOAD_CONST(arg=4, lineno=9)
DEBUG:numba.core.bytefl

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2b6b7a5c01c0>
DEBUG:numba.core.ssa:on stmt: a = arg(0, name=a)
DEBUG:numba.core.ssa:on stmt: b = arg(1, name=b)
DEBUG:numba.core.ssa:on stmt: $2load_global.0 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $4load_method.1 = getattr(value=$2load_global.0, attr=zeros)
DEBUG:numba.core.ssa:on stmt: $const6.2 = const(int, 3)
DEBUG:numba.core.ssa:on stmt: c = call $4load_method.1($const6.2, func=$4load_method.1, args=[Var($const6.2, coordinate_numba.py:6)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $const14.5 = const(int, 1)
DEBUG:numba.core.ssa:on stmt: $16binary_subscr.6 = static_getitem(value=a, index=1, index_var=$const14.5, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: $const20.8 = const(int, 2)
DEBUG:numba.core.ssa:on stm

DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_METHOD(arg=1, lineno=14)
DEBUG:numba.core.byteflow:stack ['$2load_global.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=LOAD_FAST(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1']
DEBUG:numba.core.byteflow:dispatch pc=8, inst=LOAD_FAST(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1', '$a6.2']
DEBUG:numba.core.byteflow:dispatch pc=10, inst=CALL_METHOD(arg=2, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1', '$a6.2', '$a8.3']
DEBUG:numba.core.byteflow:dispatch pc=12, inst=STORE_FAST(arg=1, lineno=14)
DEBUG:numba.core.byteflow:stack ['$10call_method.4']
D

DEBUG:numba.core.byteflow:dispatch pc=14, inst=UNPACK_SEQUENCE(arg=1, lineno=509)
DEBUG:numba.core.byteflow:stack ['$12load_attr.5']
DEBUG:numba.core.byteflow:dispatch pc=16, inst=STORE_FAST(arg=3, lineno=509)
DEBUG:numba.core.byteflow:stack ['$14unpack_sequence.6']
DEBUG:numba.core.byteflow:dispatch pc=18, inst=LOAD_FAST(arg=2, lineno=510)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=20, inst=LOAD_FAST(arg=3, lineno=510)
DEBUG:numba.core.byteflow:stack ['$m18.8']
DEBUG:numba.core.byteflow:dispatch pc=22, inst=COMPARE_OP(arg=3, lineno=510)
DEBUG:numba.core.byteflow:stack ['$m18.8', '$n20.9']
DEBUG:numba.core.byteflow:dispatch pc=24, inst=POP_JUMP_IF_FALSE(arg=34, lineno=510)
DEBUG:numba.core.byteflow:stack ['$22compare_op.10']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=26, stack=(), blockstack=(), npush=0), Edge(pc=34, stack=(), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=26 nstack_initial=0), State(pc_initial

...

DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=106)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_FAST(arg=0, lineno=106)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_FAST(arg=1, lineno=106)
DEBUG:numba.core.byteflow:stack ['$angle_position_12.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=BINARY_SUBTRACT(arg=None, lineno=1)
DEBUG:numba.core.byteflow:stack ['$angle_position_12.0', '$bond_position_24.1']
DEBUG:numba.core.byteflow:dispatch pc=8, inst=RETURN_VALUE(arg=None, lineno=1)
DEBUG:numba.core.byteflow:stack ['$6binary_subtract.2']
DEBUG:numba.core.byteflow:end state. edges=[]
DEBUG:numba.core.byteflow:-------------------------Prune PHIs-------------------------
DEBUG:numba.core.byteflow:Used_phis: defaultdict(<class 'set'>, {State(pc_initial=0 nstack_initial=0): s

DEBUG:numba.core.byteflow:keep phismap: {}
DEBUG:numba.core.byteflow:new_out: defaultdict(<class 'dict'>, {})
DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$a_12.0'}), (4, {'res': '$_32call_function_12_14.1'}), (6, {'lhs': '$a_12.0', 'rhs': '$_32call_function_12_14.1', 'res': '$6binary_true_divide.2'}), (8, {'retval': '$6binary_true_divide.2', 'castval': '$8return_value.3'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={})
DEBUG:numba.core.interpreter:label 0:
    _32call_function_12_1 = arg(0, name=_32call_function_12_1) ['_32call_function_12_1']
    a_1 = arg(1, name=a_1)                   ['a_1']
    $6binary_true_divide.2 = a_1 / _32call_function_12_1 ['$6binary_true_divide.2', '_32call_function_12_1', 'a_1']
    $8return_value.3 = cast(value=$6binary_true_divide.2) ['$6binary_true_divide.2'

DEBUG:numba.core.ssa:on stmt: c_1 = arg(1, name=c_1)
DEBUG:numba.core.ssa:on stmt: $6binary_true_divide.2 = c_1 / _56call_function_22_1
DEBUG:numba.core.ssa:on stmt: $8return_value.3 = cast(value=$6binary_true_divide.2)
DEBUG:numba.core.ssa:on stmt: return $8return_value.3
DEBUG:numba.core.ssa:defs defaultdict(<class 'list'>,
            {'$6binary_true_divide.2': [<numba.core.ir.Assign object at 0x2b6b7afa2c70>],
             '$8return_value.3': [<numba.core.ir.Assign object at 0x2b6b7af9dc10>],
             '_56call_function_22_1': [<numba.core.ir.Assign object at 0x2b6b7afa2130>],
             'c_1': [<numba.core.ir.Assign object at 0x2b6b7afa28b0>]})
DEBUG:numba.core.ssa:SSA violators set()


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.1 jupyter extensions


In order to validate the `CustomBondForce` w.r.t. the old system:
            1. set all `scale_lambda_{i}`, `interscale_lambda_{i}` to 1 (default values are 1, also)
            2. set all `lambda_{i}_bonds` to 0 (lambda value constituting old system)
            3. query self._hybrid_to_new_bond_indices, which is a dictionary of [custom_bond_force_bond_idx : new_bond_force_bond_idx] and contains all of the `unique_new_bonds`.
                iterate through the keys and set the `CustomBondForce`'s third-to-last parameter to 0 (setting the old spring constant to zero, mimicking turning off the unique new bond altogether)
                Example:
                >>> mod_hybrid_system = copy.deepcopy(htf._hybrid_system)
                >>> new_bond_force = htf._new_system_forces['HarmonicBondForce']
                >>> force_names = {force.__name__: force for force in mod_hybrid_system.getForces()}
                >>> custom_bond_force = force_names['CustomBondForce']
                >>> for hybrid_idx, new_idx in htf._hybrid_to_new_bond_indices.items():
                >>>     hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
                >>>     hybrid_params[-1][-3] *= 0.
                >>>     custom_bond_force.setBondParameters(hybrid_idx, *hybrid_params)
            4. now, it should be the case that the reduced potential of the old system's `HarmonicBondForce` (at fixed positions) should be equal to the hybrid system's `CustomBondForce`

# Alanine dipeptide in vacuum

In [2]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.4570835785460458.
INFO:geometry:	reduced angle potential = 0.054438334638956115.
INFO:geometry:	reduced angle potential = 0.037702631097144654.
INFO:geometry:	reduced angle potential = 0.0010977233780854454.
INFO:geometry:	reduced angle potential = 1.6288259151472917.
INFO:geometry:	reduced angle potential = 0.290835678457826.
INFO:geometry:	reduced angle potential = 0.0827276933308833.
INFO:geometry:	reduced angle potential = 3.0045079302256665.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle 

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478861179005435e-06
INFO:geometry:Final logp_proposal: -26892.696051443654


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO

In [3]:
# Save htf
with open("atp_vacuum.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [4]:
# Load htf
with open("atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [5]:
htf

In [6]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6b7aaaf2d0> >]

In [7]:
htf._topology_proposal.old_system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b6b7aaafc00> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b6b7aaaf810> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b6b7aaaf540> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b6b7aaaf060> >]

In [8]:
# Get harmonic bond force and old positions
old_system = htf._topology_proposal.old_system
harmonic_bond_force = old_system.getForce(0)
old_positions = htf.old_positions(htf.hybrid_positions)

In [9]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(0)
hybrid_positions = htf.hybrid_positions

In [10]:
# Check global parameters, confirm that there are no scale lambdas
for i in range(custom_bond_force.getNumGlobalParameters()):
    print(custom_bond_force.getGlobalParameterName(i))

lambda_0_bonds_old
lambda_0_bonds_new
nonscale_lambda_bonds


In [11]:
# Check per bond parameters
for i in range(custom_bond_force.getNumPerBondParameters()):
    print(custom_bond_force.getPerBondParameterName(i))

nonscale_region
environment_region
alchemical_region_0
length1
K1
length2
K2


In [12]:
# Confirm that custom bond force global parameter `lambda_{i}_bonds_old` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [13]:
# Confirm that custom bond force global parameter `lambda_{i}_bonds_new` is 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_new':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

0.0


In [14]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [15]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, new_idx in htf._hybrid_to_new_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-3] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

## Note that K1 is the spring constant for all terms in the system at lambda = 0 
## and K2 is the spring constant for all terms in the system at lambda = 1

In [16]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(22, 28, [1.0, 0.0, 1.0, 0.141, 0.0, 0.141, 267775.99999999994])

In [17]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=old_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(old_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]


In [18]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('AndersenThermostat', 0.0)]


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [169]:
# Load htf
with open("atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [170]:
# Get harmonic bond force and new positions
new_system = htf._topology_proposal.new_system
harmonic_bond_force = new_system.getForce(0)
new_positions = htf.new_positions(htf.hybrid_positions)

In [171]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(0)
hybrid_positions = htf.hybrid_positions

In [172]:
# Set `lambda_{i}_bonds_old` to 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
        custom_bond_force.setGlobalParameterDefaultValue(i, 0)

In [181]:
# Set `lambda_{i}_bonds_new` to 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_new':
        custom_bond_force.setGlobalParameterDefaultValue(i, 1)

In [176]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [182]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, old_idx in htf._hybrid_to_old_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-1] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

In [183]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(8, 10, [1.0, 0.0, 1.0, 0.1526, 259407.99999999994, 0.1526, 0.0])

In [184]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=new_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(new_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.881703491608262), ('HarmonicAngleForce', 8.64103997470652), ('PeriodicTorsionForce', 27.239469309657512), ('NonbondedForce', 517.8644869513105), ('AndersenThermostat', 0.0)]


In [185]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.8817034916082622), ('AndersenThermostat', 0.0)]


### Test one alchemical region, one scale region

In [29]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.24097974751383613.
INFO:geometry:	reduced angle potential = 0.02973741095022842.
INFO:geometry:	reduced angle potential = 0.04425414895367121.
INFO:geometry:	reduced angle potential = 0.15299989018286078.
INFO:geometry:	reduced angle potential = 0.3684390291980196.
INFO:geometry:	reduced angle potential = 0.31319547365620803.
INFO:geometry:	reduced angle potential = 1.1259822533767507.
INFO:geometry:	reduced angle potential = 0.2655445312574788.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle f

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478861179005435e-06
INFO:geometry:Final logp_proposal: -27058.565200036184


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0)

In [30]:
# Save htf
with open("atp_vacuum_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [31]:
# Load htf
with open("atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [32]:
htf

In [33]:
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]

In [34]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6b7fdfd960> >]

In [35]:
htf._topology_proposal.old_system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b6b7fdfdb10> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b6b7fdfd0f0> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b6b7fdfd750> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b6b7fdfd1b0> >]

In [36]:
# Get harmonic bond force and old positions
old_system = htf._topology_proposal.old_system
harmonic_bond_force = old_system.getForce(0)
old_positions = htf.old_positions(htf.hybrid_positions)

In [37]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(0)
hybrid_positions = htf.hybrid_positions

In [38]:
# Check global parameters, confirm that there are no scale lambdas
for i in range(custom_bond_force.getNumGlobalParameters()):
    print(custom_bond_force.getGlobalParameterName(i))

lambda_0_bonds_old
lambda_0_bonds_new
nonscale_lambda_bonds
scale_lambda_0_bonds
interscale_lambda_0_bonds


In [39]:
# Check per bond parameters
for i in range(custom_bond_force.getNumPerBondParameters()):
    print(custom_bond_force.getPerBondParameterName(i))

nonscale_region
scale_region_0
interscale_region_0
environment_region
alchemical_region_0
length1
K1
length2
K2


In [40]:
# Confirm that custom bond force global parameter `lambda_{i}_bonds` is 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [41]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [42]:
# Confirm that custom bond force global parameter `scale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'scale_lambda_0':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [43]:
# Confirm that custom bond force global parameter `interscale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'interscale_lambda_0':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [44]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, new_idx in htf._hybrid_to_new_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-3] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

## Note that K1 is the spring constant for all terms in the system at lambda = 0 
## and K2 is the spring constant for all terms in the system at lambda = 1

In [45]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(22, 28, [0.0, 1.0, 0.0, 0.0, 1.0, 0.141, 0.0, 0.141, 267775.99999999994])

In [46]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=old_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(old_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]


In [47]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context for hybrid system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for hybrid system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('AndersenThermostat', 0.0)]


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [200]:
# Load htf
with open("atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [201]:
# Get harmonic bond force and new positions
new_system = htf._topology_proposal.new_system
harmonic_bond_force = new_system.getForce(0)
new_positions = htf.new_positions(htf.hybrid_positions)

In [202]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(0)
hybrid_positions = htf.hybrid_positions

In [203]:
# Check global parameters, confirm that there are no scale lambdas
for i in range(custom_bond_force.getNumGlobalParameters()):
    print(custom_bond_force.getGlobalParameterName(i))

lambda_0_bonds_old
lambda_0_bonds_new
nonscale_lambda_bonds
scale_lambda_0_bonds
interscale_lambda_0_bonds


In [204]:
# Check per bond parameters
for i in range(custom_bond_force.getNumPerBondParameters()):
    print(custom_bond_force.getPerBondParameterName(i))

nonscale_region
scale_region_0
interscale_region_0
environment_region
alchemical_region_0
length1
K1
length2
K2


In [205]:
# Set `lambda_{i}_bonds_old` to 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
        custom_bond_force.setGlobalParameterDefaultValue(i, 0)

In [206]:
# Set `lambda_{i}_bonds_new` to 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_new':
        custom_bond_force.setGlobalParameterDefaultValue(i, 1)

In [207]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [208]:
# Confirm that custom bond force global parameter `scale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'scale_lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [209]:
# Confirm that custom bond force global parameter `interscale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'interscale_lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [210]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, old_idx in htf._hybrid_to_old_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-1] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

In [211]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(8, 10, [0.0, 0.0, 1.0, 0.0, 1.0, 0.1526, 259407.99999999994, 0.1526, 0.0])

In [212]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=new_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(new_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 2.271134613450782), ('HarmonicAngleForce', 4.2906690195038655), ('PeriodicTorsionForce', 27.033057843000552), ('NonbondedForce', 166.33535360539238), ('AndersenThermostat', 0.0)]


In [213]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.271134613450782), ('AndersenThermostat', 0.0)]


# Alanine dipeptide in solvent

In [62]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 19, 15, 13, 16, 17]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -26801.765416946495


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity

In [63]:
# Save htf
with open("atp_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [64]:
# Load htf
with open("atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [65]:
htf

In [66]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b6b88a3a8d0> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6b88a3a780> >]

In [67]:
htf._topology_proposal.old_system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b6b88a3a600> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b6b88a3a390> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b6b88a3a5d0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b6b88a3a210> >,
 <simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b6b88a3a060> >]

In [68]:
# Get harmonic bond force and old positions
old_system = htf._topology_proposal.old_system
harmonic_bond_force = old_system.getForce(0)
old_positions = htf.old_positions(htf.hybrid_positions)

In [69]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(1)
hybrid_positions = htf.hybrid_positions

In [70]:
# Check global parameters, confirm that there are no scale lambdas
for i in range(custom_bond_force.getNumGlobalParameters()):
    print(custom_bond_force.getGlobalParameterName(i))

lambda_0_bonds_old
lambda_0_bonds_new
nonscale_lambda_bonds


In [71]:
# Check per bond parameters
for i in range(custom_bond_force.getNumPerBondParameters()):
    print(custom_bond_force.getPerBondParameterName(i))

nonscale_region
environment_region
alchemical_region_0
length1
K1
length2
K2


In [72]:
# Confirm that custom bond force global parameter `lambda_{i}_bonds` is 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [73]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [74]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, new_idx in htf._hybrid_to_new_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-3] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

## Note that K1 is the spring constant for all terms in the system at lambda = 0 
## and K2 is the spring constant for all terms in the system at lambda = 1

In [75]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(1557, 1563, [1.0, 0.0, 1.0, 0.141, 0.0, 0.141, 267775.99999999994])

In [76]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=old_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(old_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -927.788883809082), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [77]:
custom_bond_force.setUsesPeriodicBoundaryConditions(True)

In [78]:
hybrid_system.usesPeriodicBoundaryConditions()

True

In [79]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [214]:
# Load htf
with open("atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [215]:
# Get harmonic bond force and new positions
new_system = htf._topology_proposal.new_system
harmonic_bond_force = new_system.getForce(0)
new_positions = htf.new_positions(htf.hybrid_positions)

In [216]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(1)
hybrid_positions = htf.hybrid_positions

In [217]:
# Set `lambda_{i}_bonds_old` to 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
        custom_bond_force.setGlobalParameterDefaultValue(i, 0)

In [218]:
# Set `lambda_{i}_bonds_new` to 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_new':
        custom_bond_force.setGlobalParameterDefaultValue(i, 1)

In [219]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [220]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, old_idx in htf._hybrid_to_old_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-1] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

In [221]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(8, 10, [1.0, 0.0, 1.0, 0.1526, 259407.99999999994, 0.1526, 0.0])

In [222]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=new_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(new_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 3.268805346044796), ('HarmonicAngleForce', 6.544633801269215), ('PeriodicTorsionForce', 27.180005749631242), ('NonbondedForce', -891.8705506222258), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [223]:
custom_bond_force.setUsesPeriodicBoundaryConditions(True)

In [224]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.268805346044804), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


### Test one alchemical region, one scale region

In [91]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 15, 13, 17, 19, 16]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -27004.161478494676


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nano

In [92]:
# Save htf
with open("atp_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [93]:
# Load htf
with open("atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [94]:
htf

In [95]:
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]

In [96]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b6ba5af7150> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6ba5af7090> >]

In [97]:
htf._topology_proposal.old_system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b6b7aaaf5a0> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b6b7aaaf180> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b6b7aaaff00> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b6b7aaaf240> >,
 <simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b6b7aaaf930> >]

In [98]:
# Get harmonic bond force and old positions
old_system = htf._topology_proposal.old_system
harmonic_bond_force = old_system.getForce(0)
old_positions = htf.old_positions(htf.hybrid_positions)

In [99]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(1)
hybrid_positions = htf.hybrid_positions

In [100]:
# Check global parameters, confirm that there are no scale lambdas
for i in range(custom_bond_force.getNumGlobalParameters()):
    print(custom_bond_force.getGlobalParameterName(i))

lambda_0_bonds_old
lambda_0_bonds_new
nonscale_lambda_bonds
scale_lambda_0_bonds
interscale_lambda_0_bonds


In [101]:
# Check per bond parameters
for i in range(custom_bond_force.getNumPerBondParameters()):
    print(custom_bond_force.getPerBondParameterName(i))

nonscale_region
scale_region_0
interscale_region_0
environment_region
alchemical_region_0
length1
K1
length2
K2


In [102]:
# Confirm that custom bond force global parameter `lambda_{i}_bonds` is 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [103]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [104]:
# Confirm that custom bond force global parameter `scale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'scale_lambda_0':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [105]:
# Confirm that custom bond force global parameter `interscale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'interscale_lambda_0':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [106]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, new_idx in htf._hybrid_to_new_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-3] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

## Note that K1 is the spring constant for all terms in the system at lambda = 0 
## and K2 is the spring constant for all terms in the system at lambda = 1

In [107]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(1557, 1563, [0.0, 1.0, 0.0, 0.0, 1.0, 0.141, 0.0, 0.141, 267775.99999999994])

In [108]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=old_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(old_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -1041.1357727405443), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [109]:
custom_bond_force.setUsesPeriodicBoundaryConditions(True)

In [110]:
hybrid_system.usesPeriodicBoundaryConditions()

True

In [111]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context for hybrid system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for hybrid system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [225]:
# Load htf
with open("atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [226]:
# Get harmonic bond force and new positions
new_system = htf._topology_proposal.new_system
harmonic_bond_force = new_system.getForce(0)
new_positions = htf.new_positions(htf.hybrid_positions)

In [227]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(1)
hybrid_positions = htf.hybrid_positions

In [228]:
# Check global parameters, confirm that there are no scale lambdas
for i in range(custom_bond_force.getNumGlobalParameters()):
    print(custom_bond_force.getGlobalParameterName(i))

lambda_0_bonds_old
lambda_0_bonds_new
nonscale_lambda_bonds
scale_lambda_0_bonds
interscale_lambda_0_bonds


In [229]:
# Check per bond parameters
for i in range(custom_bond_force.getNumPerBondParameters()):
    print(custom_bond_force.getPerBondParameterName(i))

nonscale_region
scale_region_0
interscale_region_0
environment_region
alchemical_region_0
length1
K1
length2
K2


In [230]:
# Set `lambda_{i}_bonds_old` to 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
        custom_bond_force.setGlobalParameterDefaultValue(i, 0)

In [231]:
# Set `lambda_{i}_bonds_new` to 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_new':
        custom_bond_force.setGlobalParameterDefaultValue(i, 1)

In [232]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [233]:
# Confirm that custom bond force global parameter `scale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'scale_lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [234]:
# Confirm that custom bond force global parameter `interscale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'interscale_lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [235]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, old_idx in htf._hybrid_to_old_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-1] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

In [236]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(8, 10, [0.0, 0.0, 1.0, 0.0, 1.0, 0.1526, 259407.99999999994, 0.1526, 0.0])

In [237]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=new_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(new_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.4684578607702736), ('HarmonicAngleForce', 4.80952399451105), ('PeriodicTorsionForce', 27.201127706572517), ('NonbondedForce', -1022.2262834587395), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [238]:
custom_bond_force.setUsesPeriodicBoundaryConditions(True)

In [239]:
hybrid_system.usesPeriodicBoundaryConditions()

True

In [240]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.4684578607702563), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


#### Change the global parameter value for scale lambda and make sure the energy changes

In [241]:
# Set `scale_lambda_{i}` to 0.5
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'scale_lambda_0_bonds':
        custom_bond_force.setGlobalParameterDefaultValue(i, 0.5)

In [242]:
# Confirm that custom bond force global parameter `scale_lambda_{i}` is 0.5
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'scale_lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

0.5


In [243]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.8935911937021379), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


# CDK2 transformation in solvent

In [131]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [136]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = None
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-04-30 13:20:36,554:(121.59s):root:	Detecting phases...
2021-04-30 13:20:36,557:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-04-30 13:20:36,560:(0.00s):root:No constraints will be removed
2021-04-30 13:20:36,562:(0.00s):root:No spectators
2021-04-30 13:20:36,565:(0.00s):root:			run_type is not specified; default to None
2021-04-30 13:20:36,567:(0.00s):root:	Detecting fe_type...
2021-04-30 13:20:36,569:(0.00s):root:		fe_type: repex
2021-04-30 13:20:36,571:(0.00s):root:			offline-freq not specified: default to 10.
2021-04-30 13:20:36,573:(0.00s):root:	'neglect_angles' detected: False.
2021-04-30 13:20:36,575:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-04-30 13:20:36,578:(0.00s):root:	Creating 'output'...
2021-04-30 13:20:36,588:(0.01s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-04-30 13:20:36,589:(0.00s):

2021-04-30 13:20:42,036:(0.00s):geometry:Conducting forward proposal...
2021-04-30 13:20:42,038:(0.00s):geometry:Computing proposal order with NetworkX...
2021-04-30 13:20:42,053:(0.02s):geometry:number of atoms to be placed: 1
2021-04-30 13:20:42,056:(0.00s):geometry:Atom index proposal order is [2190]
2021-04-30 13:20:42,058:(0.00s):geometry:omitted_bonds: []
2021-04-30 13:20:42,060:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-04-30 13:20:42,374:(0.31s):geometry:creating growth system...
2021-04-30 13:20:42,451:(0.08s):geometry:	creating bond force...
2021-04-30 13:20:42,454:(0.00s):geometry:	there are 27 bonds in reference force.
2021-04-30 13:20:42,457:(0.00s):geometry:	creating angle force...
2021-04-30 13:20:42,460:(0.00s):geometry:	there are 85 angles in reference force.
2021-04-30 13:20:42,464:(0.00s):geometry:	creating torsion force...
2021-04-30 13:20:42,466:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-04-30 13:20:44,629:(2.04s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-04-30 13:21:01,321:(3.03s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-04-30 13:21:05,112:(3.79s):geometry:total reduced energy added from growth system: -5.056837038117532
2021-04-30 13:21:05,115:(0.00s):geometry:final reduced energy 43.36216011253515
2021-04-30 13:21:05,117:(0.00s):geometry:sum of energies: 43.36215903557945
2021-04-30 13:21:05,120:(0.00s):geometry:magnitude of difference in the energies: 1.0769557015066766e-06
2021-04-30 13:21:05,122:(0.00s):geometry:Final logp_proposal: 11.562669373421256


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423200139534058), ('CustomBondForce', -5.1962986311777435)]


2021-04-30 13:21:05,467:(0.35s):root:	writing pickle output...
2021-04-30 13:21:05,672:(0.21s):root:	successfully dumped pickle.
2021-04-30 13:21:05,677:(0.00s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-04-30 13:21:05,679:(0.00s):root:	writing atom_mapping.png
2021-04-30 13:21:06,471:(0.79s):root:	 steps per move application: 50
2021-04-30 13:21:06,475:(0.00s):root:	trajectory prefix: out
2021-04-30 13:21:06,477:(0.00s):root:	atom selection detected: not water
2021-04-30 13:21:06,480:(0.00s):root:	no nonequilibrium detected.
2021-04-30 13:21:06,482:(0.00s):root:	cataloging HybridTopologyFactories...
2021-04-30 13:21:06,484:(0.00s):root:		phase: solvent:
2021-04-30 13:21:06,486:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-04-30 13:21:06,489:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-04-30 13:21:06,491:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptio

In [137]:
setup_options

{'protein_pdb': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_fixed_nohet.pdb',
 'ligand_file': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf',
 'old_ligand_index': 14,
 'new_ligand_index': 15,
 'forcefield_files': ['amber/protein.ff14SB.xml',
  'amber/tip3p_standard.xml',
  'amber/tip3p_HFE_multivalent.xml'],
 'small_molecule_forcefield': 'gaff-2.11',
 'small_molecule_parameters_cache': 'cdk2-cache.json',
 'pressure': 1.0,
 'temperature': 300.0,
 'solvent_padding': 9.0,
 'save_setup_pickle_as': 'fesetup_hbonds.pkl',
 'n_steps_per_move_application': 50,
 'fe_type': 'repex',
 'checkpoint_interval': 250,
 'n_cycles': 2,
 'n_states': 6,
 'n_equilibration_iterations': 0,
 'trajectory_directory': 'output',
 'trajectory_prefix': 'out',
 'atom_selection': 'not water',
 'phases': ['solvent'],
 'protocol-type': 

In [138]:
with open("cdk2_solvent.pickle", "wb") as f:
    pickle.dump(setup_dict, f)

In [139]:
htf = setup_dict['hybrid_topology_factories']['solvent']

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [244]:
# Load htf
with open("cdk2_solvent.pickle", "rb") as f:
    setup_dict = pickle.load(f)
    htf = setup_dict['hybrid_topology_factories']['solvent']

In [245]:
htf

In [246]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b6b8ac7d990> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6b8af0c180> >]

In [247]:
htf._topology_proposal.old_system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b6b80ea6690> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b6b80ea6780> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b6b80ea65d0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b6b80ea6630> >,
 <simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b6b80ea6420> >]

In [248]:
# Get harmonic bond force and old positions
old_system = htf._topology_proposal.old_system
harmonic_bond_force = old_system.getForce(0)
old_positions = htf.old_positions(htf.hybrid_positions)

In [249]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(1)
hybrid_positions = htf.hybrid_positions

In [250]:
# Check global parameters, confirm that there are no scale lambdas
for i in range(custom_bond_force.getNumGlobalParameters()):
    print(custom_bond_force.getGlobalParameterName(i))

lambda_0_bonds_old
lambda_0_bonds_new
nonscale_lambda_bonds


In [251]:
# Check per bond parameters
for i in range(custom_bond_force.getNumPerBondParameters()):
    print(custom_bond_force.getPerBondParameterName(i))

nonscale_region
environment_region
alchemical_region_0
length1
K1
length2
K2


In [148]:
# Confirm that custom bond force global parameter `lambda_{i}_bonds` is 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [149]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

In [150]:
for atom in htf.hybrid_topology.atoms:
    print(atom)

MOL1-C1
MOL1-C2
MOL1-C3
MOL1-C4
MOL1-C5
MOL1-C6
MOL1-C7
MOL1-C8
MOL1-C9
MOL1-C10
MOL1-C11
MOL1-C12
MOL1-C13
MOL1-N1
MOL1-C14
MOL1-O1
MOL1-C15
MOL1-N2
MOL1-N3
MOL1-C16
MOL1-N4
MOL1-N5
MOL1-C17
MOL1-C18
MOL1-H1
MOL1-H2
MOL1-H3
MOL1-H4
MOL1-H5
MOL1-H6
MOL1-H7
MOL1-H8
MOL1-H9
MOL1-H10
MOL1-H11
MOL1-H12
MOL1-H13
MOL1-H14
MOL1-H15
MOL1-H16
MOL1-H17
MOL1-H18
MOL1-H19
MOL1-H20
MOL1-Br1
MOL1-H15
HOH2-O
HOH2-H1
HOH2-H2
HOH3-O
HOH3-H1
HOH3-H2
HOH4-O
HOH4-H1
HOH4-H2
HOH5-O
HOH5-H1
HOH5-H2
HOH6-O
HOH6-H1
HOH6-H2
HOH7-O
HOH7-H1
HOH7-H2
HOH8-O
HOH8-H1
HOH8-H2
HOH9-O
HOH9-H1
HOH9-H2
HOH10-O
HOH10-H1
HOH10-H2
HOH11-O
HOH11-H1
HOH11-H2
HOH12-O
HOH12-H1
HOH12-H2
HOH13-O
HOH13-H1
HOH13-H2
HOH15-O
HOH15-H1
HOH15-H2
HOH16-O
HOH16-H1
HOH16-H2
HOH17-O
HOH17-H1
HOH17-H2
HOH18-O
HOH18-H1
HOH18-H2
HOH19-O
HOH19-H1
HOH19-H2
HOH20-O
HOH20-H1
HOH20-H2
HOH21-O
HOH21-H1
HOH21-H2
HOH22-O
HOH22-H1
HOH22-H2
HOH23-O
HOH23-H1
HOH23-H2
HOH24-O
HOH24-H1
HOH24-H2
HOH25-O
HOH25-H1
HOH25-H2
HOH26-O
HOH26-H1
HOH26-H2
HOH27-O
HO

HOH652-O
HOH652-H1
HOH652-H2
HOH653-O
HOH653-H1
HOH653-H2
HOH654-O
HOH654-H1
HOH654-H2
HOH655-O
HOH655-H1
HOH655-H2
HOH656-O
HOH656-H1
HOH656-H2
HOH657-O
HOH657-H1
HOH657-H2
HOH658-O
HOH658-H1
HOH658-H2
HOH659-O
HOH659-H1
HOH659-H2
HOH660-O
HOH660-H1
HOH660-H2
HOH661-O
HOH661-H1
HOH661-H2
HOH662-O
HOH662-H1
HOH662-H2
HOH663-O
HOH663-H1
HOH663-H2
HOH664-O
HOH664-H1
HOH664-H2
HOH665-O
HOH665-H1
HOH665-H2
HOH666-O
HOH666-H1
HOH666-H2
HOH667-O
HOH667-H1
HOH667-H2
HOH668-O
HOH668-H1
HOH668-H2
HOH669-O
HOH669-H1
HOH669-H2
HOH670-O
HOH670-H1
HOH670-H2
HOH671-O
HOH671-H1
HOH671-H2
HOH672-O
HOH672-H1
HOH672-H2
HOH673-O
HOH673-H1
HOH673-H2
HOH674-O
HOH674-H1
HOH674-H2
HOH675-O
HOH675-H1
HOH675-H2
HOH676-O
HOH676-H1
HOH676-H2
HOH677-O
HOH677-H1
HOH677-H2
HOH678-O
HOH678-H1
HOH678-H2
HOH679-O
HOH679-H1
HOH679-H2
HOH680-O
HOH680-H1
HOH680-H2
HOH681-O
HOH681-H1
HOH681-H2
HOH682-O
HOH682-H1
HOH682-H2
HOH683-O
HOH683-H1
HOH683-H2
HOH684-O
HOH684-H1
HOH684-H2
HOH685-O
HOH685-H1
HOH685-H2
HOH686-O
HOH68

In [151]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, new_idx in htf._hybrid_to_new_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-3] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

## Note that K1 is the spring constant for all terms in the system at lambda = 0 
## and K2 is the spring constant for all terms in the system at lambda = 1

In [152]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(8, 10, [0.0, 0.0, 1.0, 0.0, 1.0, 0.1526, 259407.99999999994, 0.1526, 0.0])

In [153]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=old_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(old_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1700.223999491089), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [154]:
custom_bond_force.setUsesPeriodicBoundaryConditions(True)

In [155]:
hybrid_system.usesPeriodicBoundaryConditions()

True

In [156]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [252]:
# Load htf
with open("cdk2_solvent.pickle", "rb") as f:
    setup_dict = pickle.load(f)
    htf = setup_dict['hybrid_topology_factories']['solvent']

In [253]:
# Get harmonic bond force and new positions
new_system = htf._topology_proposal.new_system
harmonic_bond_force = new_system.getForce(0)
new_positions = htf.new_positions(htf.hybrid_positions)

In [254]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(1)
hybrid_positions = htf.hybrid_positions

In [255]:
# Set `lambda_{i}_bonds_old` to 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
        custom_bond_force.setGlobalParameterDefaultValue(i, 0)

In [256]:
# Set `lambda_{i}_bonds_new` to 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_new':
        custom_bond_force.setGlobalParameterDefaultValue(i, 1)

In [257]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [258]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, old_idx in htf._hybrid_to_old_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-1] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

In [259]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(11, 44, [1.0, 0.0, 1.0, 0.1908, 135728.95999999996, 0.1908, 0.0])

In [260]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=new_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(new_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.16743219736821), ('PeriodicTorsionForce', 15.08569135447898), ('NonbondedForce', 1717.5107626234656), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [261]:
custom_bond_force.setUsesPeriodicBoundaryConditions(True)

In [262]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.519508183899664), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [263]:
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[44, 2197]

### Test one alchemical region, one scale region

In [264]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

In [268]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = [[44, 2197]]
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-04-30 13:28:52,175:(13.52s):root:	Detecting phases...
2021-04-30 13:28:52,179:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-04-30 13:28:52,182:(0.00s):root:No constraints will be removed
2021-04-30 13:28:52,185:(0.00s):root:No spectators
2021-04-30 13:28:52,187:(0.00s):root:			run_type is not specified; default to None
2021-04-30 13:28:52,190:(0.00s):root:	Detecting fe_type...
2021-04-30 13:28:52,193:(0.00s):root:		fe_type: repex
2021-04-30 13:28:52,195:(0.00s):root:			offline-freq not specified: default to 10.
2021-04-30 13:28:52,198:(0.00s):root:	'neglect_angles' detected: False.
2021-04-30 13:28:52,199:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-04-30 13:28:52,200:(0.00s):root:	Creating 'output'...
2021-04-30 13:28:52,210:(0.01s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-04-30 13:28:52,211:(0.00s):r

2021-04-30 13:28:55,375:(0.00s):geometry:Conducting forward proposal...
2021-04-30 13:28:55,377:(0.00s):geometry:Computing proposal order with NetworkX...
2021-04-30 13:28:55,385:(0.01s):geometry:number of atoms to be placed: 1
2021-04-30 13:28:55,387:(0.00s):geometry:Atom index proposal order is [2190]
2021-04-30 13:28:55,388:(0.00s):geometry:omitted_bonds: []
2021-04-30 13:28:55,389:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-04-30 13:28:55,532:(0.14s):geometry:creating growth system...
2021-04-30 13:28:55,570:(0.04s):geometry:	creating bond force...
2021-04-30 13:28:55,571:(0.00s):geometry:	there are 27 bonds in reference force.
2021-04-30 13:28:55,573:(0.00s):geometry:	creating angle force...
2021-04-30 13:28:55,574:(0.00s):geometry:	there are 85 angles in reference force.
2021-04-30 13:28:55,576:(0.00s):geometry:	creating torsion force...
2021-04-30 13:28:55,578:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-04-30 13:28:56,513:(0.87s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-04-30 13:29:06,219:(1.76s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-04-30 13:29:08,410:(2.19s):geometry:total reduced energy added from growth system: -5.056837038117532
2021-04-30 13:29:08,413:(0.00s):geometry:final reduced energy 43.36216011253515
2021-04-30 13:29:08,415:(0.00s):geometry:sum of energies: 43.36215903557945
2021-04-30 13:29:08,417:(0.00s):geometry:magnitude of difference in the energies: 1.0769557015066766e-06
2021-04-30 13:29:08,418:(0.00s):geometry:Final logp_proposal: 11.56594796641107


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423200139534058), ('CustomBondForce', -5.1962986311777435)]


2021-04-30 13:29:08,711:(0.29s):root:	writing pickle output...
2021-04-30 13:29:08,907:(0.20s):root:	successfully dumped pickle.
2021-04-30 13:29:08,912:(0.00s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-04-30 13:29:08,914:(0.00s):root:	writing atom_mapping.png
2021-04-30 13:29:08,980:(0.07s):root:	 steps per move application: 50
2021-04-30 13:29:08,984:(0.00s):root:	trajectory prefix: out
2021-04-30 13:29:08,986:(0.00s):root:	atom selection detected: not water
2021-04-30 13:29:08,989:(0.00s):root:	no nonequilibrium detected.
2021-04-30 13:29:08,991:(0.00s):root:	cataloging HybridTopologyFactories...
2021-04-30 13:29:08,994:(0.00s):root:		phase: solvent:
2021-04-30 13:29:08,996:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-04-30 13:29:08,998:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-04-30 13:29:09,001:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptio

In [269]:
setup_options

{'protein_pdb': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_fixed_nohet.pdb',
 'ligand_file': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf',
 'old_ligand_index': 14,
 'new_ligand_index': 15,
 'forcefield_files': ['amber/protein.ff14SB.xml',
  'amber/tip3p_standard.xml',
  'amber/tip3p_HFE_multivalent.xml'],
 'small_molecule_forcefield': 'gaff-2.11',
 'small_molecule_parameters_cache': 'cdk2-cache.json',
 'pressure': 1.0,
 'temperature': 300.0,
 'solvent_padding': 9.0,
 'save_setup_pickle_as': 'fesetup_hbonds.pkl',
 'n_steps_per_move_application': 50,
 'fe_type': 'repex',
 'checkpoint_interval': 250,
 'n_cycles': 2,
 'n_states': 6,
 'n_equilibration_iterations': 0,
 'trajectory_directory': 'output',
 'trajectory_prefix': 'out',
 'atom_selection': 'not water',
 'phases': ['solvent'],
 'protocol-type': 

In [270]:
with open("cdk2_solvent_scale_regions.pickle", "wb") as f:
    pickle.dump(setup_dict, f)

In [271]:
htf = setup_dict['hybrid_topology_factories']['solvent']

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [272]:
# Load htf
with open("cdk2_solvent_scale_regions.pickle", "rb") as f:
    setup_dict = pickle.load(f)
    htf = setup_dict['hybrid_topology_factories']['solvent']

In [273]:
htf

In [274]:
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[44, 2197]

In [275]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b6b8af0c0f0> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6ba67ece10> >]

In [276]:
htf._topology_proposal.old_system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b6b8b46bf30> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b6ba67ec870> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b6ba67ec9f0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b6ba67ec840> >,
 <simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b6ba67ec9c0> >]

In [277]:
# Get harmonic bond force and old positions
old_system = htf._topology_proposal.old_system
harmonic_bond_force = old_system.getForce(0)
old_positions = htf.old_positions(htf.hybrid_positions)

In [278]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(1)
hybrid_positions = htf.hybrid_positions

In [279]:
# Check global parameters, confirm that there are no scale lambdas
for i in range(custom_bond_force.getNumGlobalParameters()):
    print(custom_bond_force.getGlobalParameterName(i))

lambda_0_bonds_old
lambda_0_bonds_new
nonscale_lambda_bonds
scale_lambda_0_bonds
interscale_lambda_0_bonds


In [280]:
# Check per bond parameters
for i in range(custom_bond_force.getNumPerBondParameters()):
    print(custom_bond_force.getPerBondParameterName(i))

nonscale_region
scale_region_0
interscale_region_0
environment_region
alchemical_region_0
length1
K1
length2
K2


In [281]:
# Confirm that custom bond force global parameter `lambda_{i}_bonds` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [282]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [283]:
# Confirm that custom bond force global parameter `scale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'scale_lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [284]:
# Confirm that custom bond force global parameter `interscale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'interscale_lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [285]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, new_idx in htf._hybrid_to_new_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-3] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

## Note that K1 is the spring constant for all terms in the system at lambda = 0 
## and K2 is the spring constant for all terms in the system at lambda = 1

In [286]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=old_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(old_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1766.9022231456272), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [287]:
custom_bond_force.setUsesPeriodicBoundaryConditions(True)

In [288]:
hybrid_system.usesPeriodicBoundaryConditions()

True

In [289]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context for hybrid system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for hybrid system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [319]:
# Load htf
with open("cdk2_solvent_scale_regions.pickle", "rb") as f:
    setup_dict = pickle.load(f)
    htf = setup_dict['hybrid_topology_factories']['solvent']

In [320]:
# Get harmonic bond force and new positions
new_system = htf._topology_proposal.new_system
harmonic_bond_force = new_system.getForce(0)
new_positions = htf.new_positions(htf.hybrid_positions)

In [321]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(1)
hybrid_positions = htf.hybrid_positions

In [322]:
# Check global parameters, confirm that there are no scale lambdas
for i in range(custom_bond_force.getNumGlobalParameters()):
    print(custom_bond_force.getGlobalParameterName(i))

lambda_0_bonds_old
lambda_0_bonds_new
nonscale_lambda_bonds
scale_lambda_0_bonds
interscale_lambda_0_bonds


In [323]:
# Check per bond parameters
for i in range(custom_bond_force.getNumPerBondParameters()):
    print(custom_bond_force.getPerBondParameterName(i))

nonscale_region
scale_region_0
interscale_region_0
environment_region
alchemical_region_0
length1
K1
length2
K2


In [324]:
# Set `lambda_{i}_bonds_old` to 0
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
        custom_bond_force.setGlobalParameterDefaultValue(i, 0)

In [325]:
# Set `lambda_{i}_bonds_new` to 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_new':
        custom_bond_force.setGlobalParameterDefaultValue(i, 1)

In [326]:
# Confirm that custom bond force global parameter `nonscale_lambda` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'nonscale_lambda_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [327]:
# Confirm that custom bond force global parameter `scale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'scale_lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [328]:
# Confirm that custom bond force global parameter `interscale_lambda_0` is 1
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'interscale_lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

1.0


In [329]:
# Zero the unique new bonds in the custom bond force
for hybrid_idx, old_idx in htf._hybrid_to_old_bond_indices.items():
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
    hybrid_params = list(hybrid_params)
    hybrid_params[-1] *= 0
    custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

In [330]:
# Check that I zeroed the right parameter
p1, p2, hybrid_params

(11, 44, [0.0, 0.0, 1.0, 0.0, 1.0, 0.1908, 135728.95999999996, 0.1908, 0.0])

In [331]:
## Get energy components of standard bond force

platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-hybrid system
thermostate_other = ThermodynamicState(system=new_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(new_positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.31629908967661), ('PeriodicTorsionForce', 15.07821281507567), ('NonbondedForce', 1783.5471715609788), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [332]:
custom_bond_force.setUsesPeriodicBoundaryConditions(True)

In [333]:
hybrid_system.usesPeriodicBoundaryConditions()

True

In [334]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.519508183899664), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


#### Change the global parameter value for scale lambda and make sure the energy changes

In [341]:
# Load htf
with open("cdk2_solvent_scale_regions.pickle", "rb") as f:
    setup_dict = pickle.load(f)
    htf = setup_dict['hybrid_topology_factories']['solvent']

In [342]:
# Get custom bond force and hybrid positions
hybrid_system = htf.hybrid_system
custom_bond_force = hybrid_system.getForce(1)
hybrid_positions = htf.hybrid_positions

In [344]:
custom_bond_force.setUsesPeriodicBoundaryConditions(True)

In [345]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [346]:
# Set `interscale_lambda_{i}` to 0.5
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'interscale_lambda_0_bonds':
        custom_bond_force.setGlobalParameterDefaultValue(i, 0.5)

In [347]:
# Confirm that custom bond force global parameter `scale_lambda_{i}` is 0.5
for i in range(custom_bond_force.getNumGlobalParameters()):
    if custom_bond_force.getGlobalParameterName(i) == 'interscale_lambda_0_bonds':
        print(custom_bond_force.getGlobalParameterDefaultValue(i))

0.5


In [348]:
## Get energy components of custom bond force

# Create thermodynamic state for hybrid_system
thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)

# Create context forrest system
integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
context_hybrid.setPositions(hybrid_positions)

# Get energy components for rest system
components_hybrid = [component[1] for component in compute_potential_components(context_hybrid, beta=beta)]
print(compute_potential_components(context_hybrid, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.563795580927209), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


Note: There are no bonds in the custom bond force that are in scale region and there is only one in the inter region. The bond in the inter region is in the unique old bonds, so if I use the hybrid system from the previous section (without loading the hybrid system anew), I will get no change in energy after scaling interscale_lambda_0 to 0.5 because the term is already zero-ed from the last section.

In [339]:
for i in range(custom_bond_force.getNumBonds()):
    p1, p2, hybrid_params = custom_bond_force.getBondParameters(i)
    print(p1, p2, hybrid_params)

0 13 (1.0, 0.0, 0.0, 0.0, 1.0, 0.1339, 346602.55999999994, 0.1339, 346602.55999999994)
0 20 (1.0, 0.0, 0.0, 0.0, 1.0, 0.1339, 346602.55999999994, 0.1339, 346602.55999999994)
0 21 (1.0, 0.0, 0.0, 0.0, 1.0, 0.1386, 290453.27999999997, 0.1386, 290453.27999999997)
1 17 (1.0, 0.0, 0.0, 0.0, 1.0, 0.1317, 377145.7599999999, 0.1317, 377145.7599999999)
1 18 (1.0, 0.0, 0.0, 0.0, 1.0, 0.13799999999999998, 296645.5999999999, 0.13799999999999998, 296645.5999999999)
2 3 (1.0, 0.0, 0.0, 0.0, 1.0, 0.15380000000000002, 194555.99999999997, 0.15380000000000002, 194555.99999999997)
2 15 (1.0, 0.0, 0.0, 0.0, 1.0, 0.1432, 238320.63999999996, 0.1432, 238320.63999999996)
3 4 (1.0, 0.0, 0.0, 0.0, 1.0, 0.15380000000000002, 194555.99999999997, 0.15380000000000002, 194555.99999999997)
3 8 (1.0, 0.0, 0.0, 0.0, 1.0, 0.15380000000000002, 194555.99999999997, 0.15380000000000002, 194555.99999999997)
4 5 (1.0, 0.0, 0.0, 0.0, 1.0, 0.15380000000000002, 194555.99999999997, 0.15380000000000002, 194555.99999999997)
5 6 (1.0

In [340]:
for atom in htf.hybrid_topology.atoms:
    print(atom.index, atom)

0 MOL1-C1
1 MOL1-C2
2 MOL1-C3
3 MOL1-C4
4 MOL1-C5
5 MOL1-C6
6 MOL1-C7
7 MOL1-C8
8 MOL1-C9
9 MOL1-C10
10 MOL1-C11
11 MOL1-C12
12 MOL1-C13
13 MOL1-N1
14 MOL1-C14
15 MOL1-O1
16 MOL1-C15
17 MOL1-N2
18 MOL1-N3
19 MOL1-C16
20 MOL1-N4
21 MOL1-N5
22 MOL1-C17
23 MOL1-C18
24 MOL1-H1
25 MOL1-H2
26 MOL1-H3
27 MOL1-H4
28 MOL1-H5
29 MOL1-H6
30 MOL1-H7
31 MOL1-H8
32 MOL1-H9
33 MOL1-H10
34 MOL1-H11
35 MOL1-H12
36 MOL1-H13
37 MOL1-H14
38 MOL1-H15
39 MOL1-H16
40 MOL1-H17
41 MOL1-H18
42 MOL1-H19
43 MOL1-H20
44 MOL1-Br1
2197 MOL1-H15
45 HOH2-O
46 HOH2-H1
47 HOH2-H2
48 HOH3-O
49 HOH3-H1
50 HOH3-H2
51 HOH4-O
52 HOH4-H1
53 HOH4-H2
54 HOH5-O
55 HOH5-H1
56 HOH5-H2
57 HOH6-O
58 HOH6-H1
59 HOH6-H2
60 HOH7-O
61 HOH7-H1
62 HOH7-H2
63 HOH8-O
64 HOH8-H1
65 HOH8-H2
66 HOH9-O
67 HOH9-H1
68 HOH9-H2
69 HOH10-O
70 HOH10-H1
71 HOH10-H2
72 HOH11-O
73 HOH11-H1
74 HOH11-H2
75 HOH12-O
76 HOH12-H1
77 HOH12-H2
78 HOH13-O
79 HOH13-H1
80 HOH13-H2
81 HOH14-O
82 HOH14-H1
83 HOH14-H2
84 HOH15-O
85 HOH15-H1
86 HOH15-H2
87 HOH16-O
88 

1674 HOH547-O
1675 HOH547-H1
1676 HOH547-H2
1677 HOH548-O
1678 HOH548-H1
1679 HOH548-H2
1680 HOH549-O
1681 HOH549-H1
1682 HOH549-H2
1683 HOH550-O
1684 HOH550-H1
1685 HOH550-H2
1686 HOH551-O
1687 HOH551-H1
1688 HOH551-H2
1689 HOH552-O
1690 HOH552-H1
1691 HOH552-H2
1692 HOH553-O
1693 HOH553-H1
1694 HOH553-H2
1695 HOH554-O
1696 HOH554-H1
1697 HOH554-H2
1698 HOH555-O
1699 HOH555-H1
1700 HOH555-H2
1701 HOH556-O
1702 HOH556-H1
1703 HOH556-H2
1704 HOH557-O
1705 HOH557-H1
1706 HOH557-H2
1707 HOH558-O
1708 HOH558-H1
1709 HOH558-H2
1710 HOH559-O
1711 HOH559-H1
1712 HOH559-H2
1713 HOH560-O
1714 HOH560-H1
1715 HOH560-H2
1716 HOH561-O
1717 HOH561-H1
1718 HOH561-H2
1719 HOH562-O
1720 HOH562-H1
1721 HOH562-H2
1722 HOH563-O
1723 HOH563-H1
1724 HOH563-H2
1725 HOH564-O
1726 HOH564-H1
1727 HOH564-H2
1728 HOH565-O
1729 HOH565-H1
1730 HOH565-H2
1731 HOH566-O
1732 HOH566-H1
1733 HOH566-H2
1734 HOH567-O
1735 HOH567-H1
1736 HOH567-H2
1737 HOH568-O
1738 HOH568-H1
1739 HOH568-H2
1740 HOH569-O
1741 HOH569-H1
174

In [ ]:
# TO DO: actually test the scaling of the energies matches